In [ ]:
!pip install transformers[sentencepiece] sumy

In [ ]:
from transformers import pipeline
import torch
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:

    print("NLTK 'punkt' paketi bulunamadı, indiriliyor...")
    nltk.download('punkt')
    print("İndirme tamamlandı.")

In [ ]:
text = """
Küresel iklim değişikliği, dünya genelinde sıcaklıkların artmasına ve hava olaylarının değişmesine yol açan önemli bir çevre sorunudur.
Sanayi Devrimi'nden bu yana fosil yakıtların yoğun kullanımı, atmosferdeki sera gazı konsantrasyonlarını artırmıştır.
Bu gazlar, güneşten gelen ısının atmosferde hapsolmasına neden olarak dünyanın ısınmasına yol açar.
Artan sıcaklıklar buzulların erimesine, deniz seviyelerinin yükselmesine ve kuraklık, sel gibi ekstrem hava olaylarının artmasına neden olmaktadır.

Bilim insanları, karbon salımını azaltmak için yenilenebilir enerji kaynaklarının kullanılmasını,
enerji verimliliğinin artırılmasını ve ormanların korunmasını önermektedir. Ayrıca bireylerin günlük yaşamlarında enerji tasarrufu
sağlamaları ve karbon ayak izlerini azaltmaları teşvik edilmektedir. Ancak küresel işbirliği olmadan bu sorunun üstesinden gelmek oldukça zordur.
Hükümetlerin sera gazı emisyonlarını sınırlayan politikalar geliştirmesi gerekmektedir.

Son yıllarda, iklim değişikliğinin ekonomik etkileri de giderek daha görünür hale gelmiştir. Tarım sektöründe verim kayıpları yaşanırken,
sel ve fırtına gibi afetler altyapıya büyük zararlar vermektedir. Bu durum, ülkelerin ekonomik büyümesini ve gıda güvenliğini tehdit etmektedir.
İklim değişikliğine karşı uyum sağlamak ve dirençli yapılar inşa etmek, yalnızca çevresel değil,
aynı zamanda ekonomik sürdürülebilirlik açısından da kritik hale gelmiştir.
"""


In [ ]:
question1 = "İklim değişikliğinin ekonomik etkileri nelerdir?"
question2 = "Bilim insanları iklim değişikliğiyle mücadelede hangi yöntemleri öneriyor?"

print
summarizer_pipeline = pipeline("summarization", model="google/mt5-base")

prefixed_text = "özetle: " + text
summary_abstractive = summarizer_pipeline(prefixed_text, max_length=100, min_length=25, do_sample=False)
print(summary_abstractive[0]['summary_text'])
print("-" * 50)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<extra_id_0> özetle: Küresel iklim değişik... <extra_id_1>, özet özetle: <extra_id_2>, özet:
--------------------------------------------------


In [ ]:
print
parser = PlaintextParser.from_string(text, Tokenizer("turkish"))
#metindeki her cümlenin ne kadar "merkezi" ve "önemli" olduğunu matematiksel olarak hesaplar.
#Algoritmayı çalıştırır ve ondan, metnin tamamını en iyi temsil ettiğini düşündüğü en önemli 3 cümleyi seçmesini isteriz.
summarizer_lex = LexRankSummarizer()
summary_extractive = summarizer_lex(parser.document, 3)
for sentence in summary_extractive:
    print(str(sentence))
print("-" * 50)

Küresel iklim değişikliği, dünya genelinde sıcaklıkların artmasına ve hava olaylarının değişmesine yol açan önemli bir çevre sorunudur.
Sanayi Devrimi'nden bu yana fosil yakıtların yoğun kullanımı, atmosferdeki sera gazı konsantrasyonlarını artırmıştır.
Bu gazlar, güneşten gelen ısının atmosferde hapsolmasına neden olarak dünyanın ısınmasına yol açar.
--------------------------------------------------


In [ ]:
print
qa_abstractive_pipeline = pipeline("text2text-generation", model="google/mt5-base")
input_text1 = f"soru: {question1} bağlam: {text}"
input_text2 = f"soru: {question2} bağlam: {text}"
#Modeli çalıştırırız. Model, soruyu ve metni anlar, cevabı kendi içinde formüle eder ve yeni, özgün bir cevap cümlesi üretir.

answer1_abs = qa_abstractive_pipeline(input_text1, max_length=50, min_length=10)
answer2_abs = qa_abstractive_pipeline(input_text2, max_length=50, min_length=10)


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [ ]:
print(f"Soru 1: {question1}")
print(f"Cevap 1: {answer1_abs[0]['generated_text']}\n")
print(f"Soru 2: {question2}")
print(f"Cevap 2: {answer2_abs[0]['generated_text']}")
print("-" * 50)

Soru 1: İklim değişikliğinin ekonomik etkileri nelerdir?
Cevap 1: <extra_id_0>, iklim değişikliğinin ekonomik etkileri nelerdir?

Soru 2: Bilim insanları iklim değişikliğiyle mücadelede hangi yöntemleri öneriyor?
Cevap 2: <extra_id_0> Bilim insanları iklim değişikliğiyle mücadelede
--------------------------------------------------


In [ ]:
print
# Model: 'savasy/bert-base-turkish-squad' -> Ödev notlarında da önerilen, Türkçe SQuAD verisiyle eğitilmiş, bu görev için en iyi seçeneklerden biri.
qa_extractive_pipeline = pipeline("question-answering", model="savasy/bert-base-turkish-squad")

answer1_ext = qa_extractive_pipeline(question=question1, context=text)
answer2_ext = qa_extractive_pipeline(question=question2, context=text)

Some weights of the model checkpoint at savasy/bert-base-turkish-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
print(f"Soru 1: {question1}")
print(f"Cevap 1: {answer1_ext['answer']}\n")
print(f"Soru 2: {question2}")
print(f"Cevap 2: {answer2_ext['answer']}")
print("-" * 50)

Soru 1: İklim değişikliğinin ekonomik etkileri nelerdir?
Cevap 1: iklim değişikliğinin ekonomik etkileri de giderek daha görünür hale gelmiştir.

Soru 2: Bilim insanları iklim değişikliğiyle mücadelede hangi yöntemleri öneriyor?
Cevap 2: yenilenebilir enerji kaynaklarının kullanılmasını,
enerji verimliliğinin artırılmasını ve ormanların korunmasını
--------------------------------------------------
